In [ ]:
'''
Based on the Grow Tree and BestSplit-Class algorithms and using the entropy impurity function, create a decision tree to learn the GoodMovie concept. Show how each node is
decided (based on comparing impurity measures), then draw the full decision tree. If there are ties in impurity measures, give higher priority to attributes and values according to their order on page 1: that is, Budget is the highest priority feature and Director is the lowest priority; within Budget, Low is the highest priority value, followed
by Medium and then High. [Normally these might be randomly chosen, but we'll use this
"inductive bias."] If there are ties to determine a leaf's label, give priority to "Yes." Now apply this learned concept (decision tree) to the three test data sets posted on the Assignments page, and list the correctly and incorrectly classified examples (by number),
for each test data set.
What is the error rate of the decision tree on the training data? On each test data set?
'''

In [4]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns 
from IPython.display import Image
import pydotplus

In [5]:
# Read training.txt
df = pd.read_csv("training.txt", sep="\t", header=None)
df.columns = ["Budget", "Genre", "FamousActors", "Directors", "GoodMovie"]
df

# Calculate the entropy of the GoodMovie column
def entropy(col):
    elements, counts = np.unique(col, return_counts=True)
    entropy = np.sum([(-counts[i]/np.sum(counts)) * math.log(counts[i]/np.sum(counts), 2) for i in range(len(elements))])
    return entropy

# Calculate the information gain
def InfoGain(data, split_attribute_name, target_name="GoodMovie"):
    # Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    # Calculate the values and the corresponding counts for the split attribute 
    vals, counts= np.unique(data[split_attribute_name], return_counts=True)
    
    # Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts)) * entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    # Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

# Find the best attribute to split on
def BestSplit(data, attributes, target_name="GoodMovie"):
    # Calculate the information gain for each attribute
    gains = [InfoGain(data, attr, target_name) for attr in attributes]
    # Return the attribute with the highest information gain
    return attributes[np.argmax(gains)]

# Create a new node
def CreateNode(data, attributes, node_type, label=None):
    # Create and return a tree node
    return {"data": data, "attributes": attributes, "node_type": node_type, "label": label}

# Create a leaf node
def CreateLeaf(label):
    # Create and return a leaf node
    return {"node_type": "leaf", "label": label}

# Grow the tree
def GrowTree(node, max_depth, current_depth=0):
    # Splitting criteria
    # 1. All the tuples belong to the same classification
    # 2. There are no more remaining attributes
    # 3. Maximum tree depth has been reached
    if len(np.unique(node["data"]["GoodMovie"])) <= 1 or len(node["attributes"]) ==0 or current_depth == max_depth:
        # Create a leaf node
        leaf_label = np.unique(node["data"]["GoodMovie"])[0]
        return CreateLeaf(leaf_label)
    
    # Find the best attribute to split on
    best_attribute = BestSplit(node["data"], node["attributes"])
    
    # Create a new internal node
    new_node = CreateNode(node["data"], node["attributes"], node_type="internal", label=best_attribute)
    
    # Remove the best attribute from the attribute list
    new_node["attributes"].remove(best_attribute)
    
    # Create a new decision tree/sub-tree for each of the values in the best attribute field
    for value in np.unique(node["data"][best_attribute]):
        # Split the dataset along the value of the best attribute
        sub_data = node["data"].where(node["data"][best_attribute] == value).dropna()
        
        # Add the new decision tree/sub-tree to the empty dictionary
        new_node[value] = GrowTree(CreateNode(sub_data, new_node["attributes"], node_type="internal"), max_depth, current_depth+1)
    
    return new_node

# Print the tree
def PrintTree(node, spacing=""):
    # Base case: we've reached a leaf
    if node["node_type"] == "leaf":
        print (spacing + "Predict", node["label"])
        return
    
    # Print the attribute at this node
    print (spacing + node["label"])
    
    # Print each subtree
    for value in node.keys():
        if value not in ["data", "attributes", "node_type", "label"]:
            print (spacing + '--> ' + value + ':')
            PrintTree(node[value], spacing + "  ")

# Classify a new instance
def Classify(instance, tree):
    # Base case: we've reached a leaf
    if tree["node_type"] == "leaf":
        return tree["label"]
    
    # Decide whether to follow the true-branch or the false-branch
    attribute_value = instance[tree["label"]]
    
    # Check if the attribute value is unknown
    if attribute_value not in tree.keys():
        return "Yes"
    
    # Follow the branch corresponding to the attribute value
    # Return the label of the leaf node
    subtree = tree[attribute_value]
    return Classify(instance, subtree)

# Classify the training set
def ClassifyAll(training_set, tree):
    # Classify each instance in the training set
    prediction = training_set.apply(Classify, args=(tree,), axis=1)
    
    # Compute the accuracy
    accuracy = np.sum(prediction == training_set["GoodMovie"]) / len(training_set["GoodMovie"])
    return accuracy

# Plot the tree
def PlotTree(tree, feature_names):
    # Create DOT data
    dot_data = export_graphviz(tree, out_file=None, feature_names=feature_names, class_names=["No", "Yes"], filled=True, rounded=True, special_characters=True)
    
    # Draw graph
    graph = pydotplus.graph_from_dot_data(dot_data)
    
    # Show graph
    Image(graph.create_png())

# Create a new tree
tree = GrowTree(CreateNode(df, df.columns[:-1], node_type="internal"), max_depth=3)

# Print the tree
PrintTree(tree)

# Plot the tree
PlotTree(tree, df.columns[:-1])




ValueError: Length mismatch: Expected axis has 1 elements, new values have 5 elements